In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [101]:
test = pd.read_csv("testdata.csv")
test.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,16,289.528046,272.688934,0.971786,292.766876,262.972504,0.953193,286.289246,262.972504,0.938451,...,467.017822,0.607089,454.707581,479.973083,0.578556,419.080627,467.017822,0.434467,3.017755,1
1,487,273.960602,168.503128,0.763964,273.526581,166.803040,0.543151,273.331268,167.011505,0.811976,...,233.254318,0.636083,265.484680,230.943604,0.626216,266.254913,230.943604,0.746555,2.204191,11
2,397,330.616425,189.445602,0.926496,338.435852,186.839111,0.955382,335.829376,184.232635,0.950043,...,421.422394,0.873018,372.320129,444.880737,0.900309,364.500671,429.241852,0.708210,3.100052,7
3,81,111.103409,453.025024,0.897365,106.167488,453.025024,0.868608,106.167488,462.896851,0.919028,...,335.528198,0.644610,341.605164,364.729950,0.570308,327.406281,360.036316,0.620297,2.121012,3
4,107,116.744026,484.343811,0.745184,105.245277,480.510895,0.809517,109.078194,484.343811,0.790606,...,480.510895,0.738209,396.546875,465.179230,0.766168,396.546875,465.179230,0.746109,2.055489,11


In [102]:
test = test.drop(["Unnamed: 0"],axis = 1)

In [115]:
temp_df = pd.read_csv("weights.csv")
class_weight = temp_df[['80', 'class_weight']].to_dict()['class_weight']


In [106]:
Xtest = test.loc[:, test.columns != '80' ].copy()

In [116]:
Xtest

,1,2,3,4,5,6,7,8,9,10,...,70,71,72,73,74,75,76,77,78,79
0,289.528046,272.688934,0.971786,292.766876,262.972504,0.953193,286.289246,262.972504,0.938451,312.199738,...,399.647736,467.017822,0.607089,454.707581,479.973083,0.578556,419.080627,467.017822,0.434467,3.017755
1,273.960602,168.503128,0.763964,273.526581,166.803040,0.543151,273.331268,167.011505,0.811976,262.869934,...,273.957336,233.254318,0.636083,265.484680,230.943604,0.626216,266.254913,230.943604,0.746555,2.204191
2,330.616425,189.445602,0.926496,338.435852,186.839111,0.955382,335.829376,184.232635,0.950043,351.468262,...,333.222900,421.422394,0.873018,372.320129,444.880737,0.900309,364.500671,429.241852,0.708210,3.100052
3,111.103409,453.025024,0.897365,106.167488,453.025024,0.868608,106.167488,462.896851,0.919028,106.167488,...,323.117950,335.528198,0.644610,341.605164,364.729950,0.570308,327.406281,360.036316,0.620297,2.121012
4,116.744026,484.343811,0.745184,105.245277,480.510895,0.809517,109.078194,484.343811,0.790606,105.245277,...,404.212708,480.510895,0.738209,396.546875,465.179230,0.766168,396.546875,465.179230,0.746109,2.055489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,425.073456,249.862961,0.913008,432.943909,236.745529,0.945416,411.956024,236.745529,0.968380,385.721161,...,380.474182,470.235840,0.072321,359.486298,467.612366,0.165087,364.733246,467.612366,0.330250,2.858833
97,438.390900,198.880692,0.844707,440.873718,191.432297,0.840236,438.390900,191.432297,0.719844,448.322113,...,438.390900,399.987396,0.860342,510.392090,412.401398,0.910459,465.701691,407.435791,0.967902,3.188279
98,278.044067,184.040207,0.860593,277.482605,181.233002,0.860579,275.236847,181.233002,0.901738,266.253815,...,273.271820,234.850479,0.072354,264.569519,231.762558,0.080878,276.640442,234.850479,0.077889,1.683751
99,107.982513,477.454651,0.554935,104.995613,474.467743,0.689109,104.995613,480.441528,0.657414,107.982513,...,354.401428,464.013611,0.068239,354.401428,469.987396,0.110412,354.401428,464.013611,0.093477,1.876813


In [107]:
model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            class_weights=class_weight,
            learning_rate=0.07964,
            reg_lambda=0.95,
            bootstrap_type='Poisson',
            subsample=0.81,
            max_depth=7, 
            grow_policy='Lossguide',
            min_data_in_leaf=10, 
            max_leaves=152,
            task_type='GPU',
            verbose=0
        )


In [108]:
model.load_model("catboostmodel.cbm")

In [109]:
y_pred = model.predict(Xtest.values)

In [110]:
target=test['80']-1

In [111]:
confusion_matrix(target,y_pred)

array([[ 2,  0,  0,  0,  0,  0,  1,  0,  0,  1],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 45,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  7,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  6,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  2,  0],
       [ 1,  0,  0,  0,  0,  2,  0,  0,  0, 23]], dtype=int64)

In [112]:
y_test=target
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
Raccuracy = accuracy_score(y_test,y_pred)
print('Accuracy: %f' % Raccuracy)
# precision tp / (tp + fp)
Rprecision = precision_score(y_test,y_pred,average='macro')
print('Precision: %f' % Rprecision)
# recall: tp / (tp + fn)
Rrecall = recall_score(y_test,y_pred,average='macro')
print('Recall: %f' % Rrecall)
# f1: 2 tp / (2 tp + fp + fn)
Rf1 = f1_score(y_test,y_pred,average='macro')
print('F1 score: %f' % Rf1)

Accuracy: 0.930693
Precision: 0.941837
Recall: 0.900962
F1 score: 0.918102


In [113]:
#fall / Not Fall detection accuracy
o_testdata=[]
for i in target:
  if(i<5):
    o_testdata.append(0)
  else:
    o_testdata.append(1)
o_preddata=[]  
for i in y_pred:
  if(i<5):
    o_preddata.append(0)
  else:
    o_preddata.append(1)


In [114]:
y_test,y_pred=o_testdata,o_preddata
Raccuracy = accuracy_score(y_test,y_pred)
print('Accuracy: %f' % Raccuracy)
# precision tp / (tp + fp)
Rprecision = precision_score(y_test,y_pred)
print('Precision: %f' % Rprecision)
# recall: tp / (tp + fn)
Rrecall = recall_score(y_test,y_pred)
print('Recall: %f' % Rrecall)
# f1: 2 tp / (2 tp + fp + fn)
Rf1 = f1_score(y_test,y_pred)
print('F1 score: %f' % Rf1)

Accuracy: 0.960396
Precision: 0.966292
Recall: 0.988506
F1 score: 0.977273
